<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Elementary-operator-matrix" data-toc-modified-id="Elementary-operator-matrix-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Elementary operator matrix</a></span></li><li><span><a href="#Gauss-transformations" data-toc-modified-id="Gauss-transformations-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gauss transformations</a></span></li><li><span><a href="#LU-decomposition" data-toc-modified-id="LU-decomposition-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>LU decomposition</a></span></li><li><span><a href="#Matrix-inversion" data-toc-modified-id="Matrix-inversion-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Matrix inversion</a></span></li><li><span><a href="#Pivoting" data-toc-modified-id="Pivoting-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Pivoting</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#Further-reading" data-toc-modified-id="Further-reading-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Further reading</a></span></li><li><span><a href="#Acknowledgment" data-toc-modified-id="Acknowledgment-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Acknowledgment</a></span></li></ul></div>

In [1]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i7-10700K CPU @ 3.80GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


In [2]:
using Pkg
Pkg.activate("../..")
Pkg.status()

  Activating environment at `~/opt/Project.toml`


      Status `~/opt/Project.toml`
  [6e4b80f9] BenchmarkTools v1.2.0
  [f65535da] Convex v0.14.16
  [a93c6f00] DataFrames v1.2.2
  [31c24e10] Distributions v0.24.18
  [c91e804a] Gadfly v1.3.3
  [bd48cda9] GraphRecipes v0.5.7
  [916415d5] Images v0.24.1
  [d330b81b] PyPlot v2.10.0
  [6f49c342] RCall v0.13.12
  [2f01184e] SparseArrays
  [10745b16] Statistics


# Gaussian Elimination and LU Decomposition

* Goal: solve linear equation
$$
\mathbf{A} \mathbf{x} = \mathbf{b}.
$$
For simplicity we consider a square matrix $\mathbf{A} \in \mathbb{R}^{n \times n}$.

* [History](https://en.wikipedia.org/wiki/Gaussian_elimination#History): the method is named after Carl Friedrich Gauss (1777–1855), although it stems from the notes of Isaac Newton. If fact, GE was known to [Chinese mathematicians](https://en.wikipedia.org/wiki/The_Nine_Chapters_on_the_Mathematical_Art) as early as 179 AD.


* A [toy example](./gelu.pdf).

In [106]:
A = [2.0 -4.0 2.0; 4.0 -9.0 7.0; 2.0 1.0 3.0]

3×3 Matrix{Float64}:
 2.0  -4.0  2.0
 4.0  -9.0  7.0
 2.0   1.0  3.0

In [4]:
b = [6.0, 20.0, 14.0]

3-element Vector{Float64}:
  6.0
 20.0
 14.0

In [5]:
# Julia way to solve linear equation
# equivalent to `solve(A, b)` in R
A \ b

3-element Vector{Float64}:
 2.000000000000001
 1.0
 2.9999999999999996

What happens when we call `A \ b` to solve a linear equation?

## Elementary operator matrix

* **Elementary operator matrix** is the identity matrix with the 0 in position $(j,k)$ replaced by $c$
$$
    \mathbf{E}_{jk}(c) = \begin{pmatrix}
    1 & & \\
    & \ddots & \\
    & & 1 & \\
    & & & \ddots & \\
    & & c & & 1 & \\
    & & & & & \ddots \\
    & & & & & & 1
    \end{pmatrix} = \mathbf{I} + c \mathbf{e}_j \mathbf{e}_k^T.
$$

* $\mathbf{E}_{jk}(c)$ is unit triangular, full rank, and its inverse is
$$
    \mathbf{E}_{jk}^{-1}(c) = \mathbf{E}_{jk}(-c).
$$

* $\mathbf{E}_{jk}(c)$ left-multiplies an $n \times m$ matrix $\mathbf{X}$ effectively replace its $j$-th row $\mathbf{x}_{j\cdot}$ by $c \mathbf{x}_{k \cdot} + \mathbf{x}_{j \cdot}$
$$
    \mathbf{E}_{jk}(c) \times \mathbf{X} = \mathbf{E}_{jk}(c) \times \begin{pmatrix}
    & & \\
    \cdots & \mathbf{x}_{k\cdot} & \cdots \\
    & & \\
    \cdots & \mathbf{x}_{j\cdot} & \cdots \\
    & & 
    \end{pmatrix} = \begin{pmatrix}
    & & \\
    \cdots & \mathbf{x}_{k\cdot} & \cdots \\
    & & \\
    \cdots & c \mathbf{x}_{k\cdot} + \mathbf{x}_{j\cdot} & \cdots \\
    & & 
    \end{pmatrix}.
$$
$2m$ flops (why?).

* Gaussian elimination applies a sequence of elementary operator matrices to transform the linear system $\mathbf{A} \mathbf{x} = \mathbf{b}$ to an upper triangular system
$$
\begin{eqnarray*}
    \mathbf{E}_{n,n-1}(c_{n,n-1}) \cdots \mathbf{E}_{21}(c_{21}) \mathbf{A} \mathbf{x} &=& \mathbf{E}_{n,n-1}(c_{n,n-1}) \cdots \mathbf{E}_{21}(c_{21}) \mathbf{b} \\
    \mathbf{U} \mathbf{x} &=& \mathbf{b}_{\text{new}}.
\end{eqnarray*}
$$
    
    Column 1:

In [ ]:
# 3×3 Matrix{Float64}: A
#  2.0  -4.0  2.0
#  4.0  -9.0  7.0
#  2.0   1.0  3.0

In [6]:
E21 = [1.0 0.0 0.0; -2.0 1.0 0.0; 0.0 0.0 1.0]

3×3 Matrix{Float64}:
  1.0  0.0  0.0
 -2.0  1.0  0.0
  0.0  0.0  1.0

In [7]:
# zero (2, 1) entry
E21 * A   # Step 1, A'

3×3 Matrix{Float64}:
 2.0  -4.0  2.0
 0.0  -1.0  3.0
 2.0   1.0  3.0

In [8]:
E31 = [1.0 0.0 0.0; 0.0 1.0 0.0; -1.0 0.0 1.0]

3×3 Matrix{Float64}:
  1.0  0.0  0.0
  0.0  1.0  0.0
 -1.0  0.0  1.0

In [9]:
# zero (3, 1) entry
E31 * E21 * A  # Step 2, A''

3×3 Matrix{Float64}:
 2.0  -4.0  2.0
 0.0  -1.0  3.0
 0.0   5.0  1.0

    Column 2:

In [10]:
E32 = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 5.0 1.0]

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  5.0  1.0

In [11]:
# zero (3, 2) entry
E32 * E31 * E21 * A   # Step 3, A'''

3×3 Matrix{Float64}:
 2.0  -4.0   2.0
 0.0  -1.0   3.0
 0.0   0.0  16.0

## Gauss transformations

* For the first column,
$$
    \mathbf{M}_1 = \mathbf{E}_{n1}(c_{n1}) \cdots \mathbf{E}_{31}(c_{31}) \mathbf{E}_{21}(c_{21}) = \begin{pmatrix}
    1 & \\
    c_{21} & \\
    & \ddots & \\
    c_{n1} & & 1
    \end{pmatrix}
$$  
For the $k$-th column,
$$
	\mathbf{M}_k = \mathbf{E}_{nk}(c_{nk}) \cdots \mathbf{E}_{k+1,k}(c_{k+1,k}) = \begin{pmatrix}
	1 & \\
	& \ddots \\
	& & 1 & \\
	& & c_{k+1,k} & 1\\
	& & \vdots & & \ddots \\
	& &  c_{n,k} & & & 1
	\end{pmatrix}.
$$

* $\mathbf{M}_1, \ldots, \mathbf{M}_{n-1}$ are called the **Gauss transformations**.

In [ ]:
# 첫 번째 행으로 나머지 행 원소들 없애는 느낌

In [114]:
M1 = E31 * E21   # inv(L2 * L1)

3×3 Matrix{Float64}:
  1.0  0.0  0.0
 -2.0  1.0  0.0
 -1.0  0.0  1.0

In [113]:
A = [1 2 3; 4 5 6; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [115]:
M1*A

3×3 Matrix{Float64}:
 1.0  2.0  3.0
 2.0  1.0  0.0
 6.0  6.0  6.0

In [13]:
M2 = E32    # inv(L3)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  5.0  1.0

* Gauss transformations $\mathbf{M}_k$ are unit triangular, full rank, with inverse
$$
	\mathbf{M}_k^{-1} = \mathbf{E}_{k+1,k}^{-1}(c_{k+1,k}) \cdots \mathbf{E}_{nk}^{-1}(c_{nk}) = \begin{pmatrix}
	1 & \\
	& \ddots \\
	& & 1 & \\
	& & - c_{k+1,k}\\
	& & \vdots & & \ddots \\
	& & - c_{n,k} & & & 1
	\end{pmatrix}.
$$

In [14]:
inv(M1)    # L2 * L1. inv(A) give the inverse of A, but use with caution (see below)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 2.0  1.0  0.0
 1.0  0.0  1.0

In [15]:
inv(M2)    # L3

3×3 Matrix{Float64}:
 1.0   0.0  0.0
 0.0   1.0  0.0
 0.0  -5.0  1.0

## LU decomposition

Gaussian elimination does

$$
\mathbf{A} x=b
$$
에서 

$$
\mathbf{M}_{n-1} \cdots \mathbf{M}_1 \mathbf{A} = \mathbf{U}.
$$
이용해서

$$
\begin{align}
    \mathbf{A} &= \mathbf{M}_1^{-1} \cdots \mathbf{M}_{n-1}^{-1} \mathbf{U} \\
      &= \mathbf{L}\mathbf{U}
\end{align}
$$

Let

\begin{equation*}
\mathbf{L} = \mathbf{M}_1^{-1} \cdots \mathbf{M}_{n-1}^{-1} = \begin{pmatrix}  
	1 & & & & \\
	- c_{21} & \ddots & & & \\
	& & 1 & & \\
	- c_{k+1,1} & & - c_{k+1,k} & & \\
	& & \vdots & & \ddots \\
	- c_{n1} & & - c_{nk} & & & 1
	\end{pmatrix}.
\end{equation*}
Thus we have the **LU decomposition**
$$
\mathbf{A} = \mathbf{L} \mathbf{U},
$$
where $\mathbf{L}$ is **unit** lower triangular and $\mathbf{U}$ is upper triangular.

* LU decomposition exists if the principal sub-matrix $\mathbf{A}[1:k, 1:k]$ is non-singular for $k=1,\ldots,n-1$.

* If the LU decomposition exists and $\mathbf{A}$ is non-singular, then the LU decmpositon is unique and
$$
    \det(\mathbf{A}) = \det(\mathbf{L}) \det(\mathbf{U}) = \prod_{k=1}^n u_{kk}.
$$
    - This is basically how Julia, R, and MATLAB compute determinants.

In [ ]:
# A[1:k, 1:k], k=1,...,n-1 가 모두 역행렬이 존재하면, LU 존재.
# LU가 존재하고, A가 정칙(non-singular), LU는 유일하게 존재.

# \= : updating operator

In [104]:
A = [2.0 -4.0 2.0; 4.0 -9.0 7.0; 2.0 1.0 3.0]
A

3×3 Matrix{Float64}:
 2.0  -4.0  2.0
 4.0  -9.0  7.0
 2.0   1.0  3.0

In [35]:
A = [2.0 -4.0 2.0; 4.0 -9.0 7.0; 2.0 1.0 3.0]
n=3
for k=1:n-1
    for i=k+1:n
        A[i, k] /= A[k, k]    # -c_ij (new a_ij = l_ij) 계산
        for j=k+1:n
            A[i, j] -= A[i, k] * A[k, j] # k가 증가하면서 ( new a_ij = u_ij) 완성된다.
        end
    end
end
A

3×3 Matrix{Float64}:
 2.0  -4.0   2.0
 2.0  -1.0   3.0
 1.0  -5.0  16.0

In [3]:
L = [1 0 0; 2 1 0; 1 -5 1]
U = [2.0 -4.0 2.0; 0  -1.0   3.0;  0  0  16.0]
L*U

3×3 Matrix{Float64}:
 2.0  -4.0  2.0
 4.0  -9.0  7.0
 2.0   1.0  3.0

* The whole LU algorithm is done *in place*, i.e., $\mathbf{A}$ is **overwritten** by $\mathbf{L}$ and $\mathbf{U}$.
```Julia
for k=1:n-1
    for i=k+1:n
        A[i, k] /= A[k, k]
        for j=k+1:n
            A[i, j] -= A[i, k] * A[k, j]
        end
    end
end
```

* The LU decomposition costs
$$
    2(n-1)^2 + 2(n-2)^2 + \cdots + 2 \cdot 1^2 \approx \frac 23 n^3 \quad \text{flops}.
$$

<img src="http://www.netlib.org/utk/papers/factor/_25826_figure159.gif" width="500" align="center"/>

Source: <http://www.netlib.org>

* Actual implementations can differ: outer product LU ($kij$ loop), block outer product LU (higher level-3 fraction), Crout's algorithm ($jki$ loop).

* Given LU decomposition $\mathbf{A} = \mathbf{L} \mathbf{U}$, solving $(\mathbf{L} \mathbf{U}) \mathbf{x} = \mathbf{b}$ for one right hand side costs $2n^2$ flops:
    - One forward substitution ($n^2$ flops) to solve
    $$
    \mathbf{L} \mathbf{y} = \mathbf{b}
    $$
    - One back substitution ($n^2$ flops) to solve
    $$
    \mathbf{U} \mathbf{x} = \mathbf{y}
    $$
    
* Therefore to solve $\mathbf{A} \mathbf{x} = \mathbf{b}$ via LU, we need a total of
$$
    \frac 23 n^3 + 2n^2 \quad \text{flops}.
$$

* If there are multiple right hand sides, LU only needs to be done once.

In [ ]:
# Ax=b 를 푸는 cost

# Ax=b1, Ax=b2, ... 여러 개를 풀어야하는 상황이라도, LU decomposition은 한번만 해주면 된다.
    # Matrix inversion

## Matrix inversion

$$A X = I$$, $A$ is invertible ==> $X=A^{-1}$, 을 푸는 것은

$$I = [e_1, e_2, \dotsc, e_n], \quad X=[x_1, x_2, \dotsc, x_n]$$

$$A x_1 = e_1, \quad A x_2 = e_2, \quad \dotsc, A x_n = e_n$$
의 여러개의 방정식을 푸는 것 과 같다.

* For matrix inversion, there are $n$ right hand sides $\mathbf{e}_1, \ldots, \mathbf{e}_n$. Taking advantage of 0s reduces $2n^3$ flops to $\frac 43 n^3$ flops. So **matrix inversion via LU** costs
$$
\frac 23 n^3 + \frac 43 n^3 = 2n^3 \quad \text{flops}.
$$

* **No inversion mentality**:  
    > **Whenever we see matrix inverse, we should think in terms of solving linear equations.**
    > In short, no ```inv(A)```.

ex) $A^{-1}b ==>$ solve $Ax=b$, `A \ b`, NOT `inv(A) *  b`. Flop counts : $\frac23 n^3 + 2n^2$ vs $2n^3 + n^2$

`A \ b` : LU decomposition 후 해를 구하는 방법.

보통, inverse matrix 가 나오면 단독으로 나오지 않고, 뒤에 행렬, 벡터가 붙어있음. 그럼 linear equation으로 생각할 수 있다. 

    We do not compute matrix inverse unless  
    1. it is necessary to compute standard errors 
(이 때는 $\Sigma^{-1}$ 이 필요함)
    
    2. number of right hand sides is much larger than $n$ 
(풀어야 할 $e_i$가 n보다 많으면)

    3. $n$ is small
 

In [ ]:
############################################################################
# 역행렬을 구해야 할 땐, inv(A)를 떠올리는 것이 아니라, linear equations 을 푼다고 생각  #
############################################################################

## Pivoting    

* Let
$$
    \mathbf{A} = \begin{bmatrix}
    0 & 1 \\
    1 & 0 \\
    \end{bmatrix}.
$$
Is there a solution to $\mathbf{A} \mathbf{x} = \mathbf{b}$ for an arbitrary $\mathbf{b}$? Does GE/LU work for $\mathbf{A}$?

In [41]:
# A is invertable, but doesn't exist LU decomposition.
A = [0.001 1; 1 1]; b=[1,2]
A\b

2-element Vector{Float64}:
 1.001001001001001
 0.998998998998999

* What if, during LU procedure, the **pivot** $a_{kk}$ is 0 or nearly 0 due to underflow?  
     **Example**:
$$
\begin{split}
0.001x_1 + x_2 &= 1  \\
     x_1 + x_2 &= 2 
\end{split}
$$
with solution

$$
\begin{bmatrix} x_1 \\ x_ 2 \end{bmatrix}
=
\begin{bmatrix} 1.001001 \\ 0.998999 \end{bmatrix}
\approx
\begin{bmatrix} 1.0 \\ 1.0 \end{bmatrix}
.
$$
유효숫자 2개인 상황 생각

* With two significant digits, GE yields 
$$
\begin{split}
0.001x_1 + x_2 &= 1  \\
     (1-1000)x_2 &= 2 - 1000 
\end{split}
$$
or (유효숫자 2개, 998 998 가 1000으로 반올림 됨)
$$
\begin{split}
0.001x_1 + x_2 &= 1 \\
     -1000 x_2 &= -1000 
\end{split}
,
$$
yielding
$$
\begin{bmatrix} x_1 \\ x_ 2 \end{bmatrix}
=
\begin{bmatrix} 0.0 \\ 1.0 \end{bmatrix}
!
$$

In [42]:
# (1,1) 성분이 꼭 0이 아니더라도, 0에 가까운 작은 수라면 부정확한 해를 구할 수 있다.
    # 0에 가까운 수 일때, LU 존재하지 않는데 억지로 하는 경우.
    # floating point system을 사용하기 때문.

* Solution: pivoting.

$$
\begin{split}
     x_1 + x_2 &= 2  \\
0.001x_1 + x_2 &= 1  
\end{split}
$$

* With two significant digits, GE yields 

$$
\begin{split}
     x_1 + x_2 &= 2  \\
     (1-0.001)x_2 &= 1 - 0.002 
\end{split}
$$

or

$$
\begin{split}
     x_1 + x_2 &= 2  \\
     1.0 x_2 &= 1.0
\end{split}
,
$$

yielding

$$
\begin{bmatrix} x_1 \\ x_ 2 \end{bmatrix}
=
\begin{bmatrix} 1.0 \\ 1.0 \end{bmatrix}
.
$$



In [ ]:
# pivoting : 식의 순서를 바꾸는 것
    # pivoting을 하면, LU decomposition이 가능해진다.

* **Partial pivoting**: before zeroing the $k$-th column, move the row with $\max_{i=k}^n |a_{ik}|$ is into the $k$-th row (called GEPP).

* LU with partial pivoting yields
$$
	\mathbf{P} \mathbf{A} = \mathbf{L} \mathbf{U},
$$
where $\mathbf{P}$ is a permutation matrix, $\mathbf{L}$ is unit lower triangular with $|\ell_{ij}| \le 1$, and $\mathbf{U}$ is upper triangular.

* Complete pivoting (GECP): Do both row and column interchanges so that the largest entry in the submatrix `A[k:n, k:n]` is permuted to the $(k,k)$-th entry. This yields the decomposition 
$$
\mathbf{P} \mathbf{A} \mathbf{Q} = \mathbf{L} \mathbf{U},
$$
where $|\ell_{ij}| \le 1$.

* GEPP is the most commonly used methods for solving **general** linear systems. GECP is more  stable but costs more computation. Partial pivoting is stable most of times (which is partially because GEPP guarantees $|\ell_{ij}| \le 1$).

$a_{kk}$ 가 0이라면, $k^{th}$ column에서 $a_{kk}$ 아래에 있는 수 $a_{(k+1)k}, a_{(k+2)k}, \dotsc, a_{nk} $ 중 절대값이 가장 큰 행과 위치를 교환한다.

In [ ]:
### Partial pivoting
# PA 에 대해 LU decomposition 을 진행한다.
# l_ij 성분들의 절대값이 1 이하가 된다.
# 일반적으로 많은 시나리오에서 cost가 더 작음.

### Complete pivoting
# 행뿐만 아니라 '열'까지 교체한다.

## Implementation

* LAPACK: [`?GETRF`](http://www.netlib.org/lapack/explore-html/dd/d9a/group__double_g_ecomputational_ga0019443faea08275ca60a734d0593e60.html#ga0019443faea08275ca60a734d0593e60) does $\mathbf{P} \mathbf{A} = \mathbf{L} \mathbf{U}$ (LU decomposition with partial pivoting) in place.

* R: `solve()` implicitly performs LU decomposition (wrapper of LAPACK routine `DGESV`). `solve()` allows specifying a single or multiple right hand sides. If none, it computes the matrix inverse. The `matrix` package contains `lu()` function that outputs `L`, `U`, and `pivot`.

* Julia: 
    - [`LinearAlgebra.lu`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.lu).
    - [`LinearAlgebra.lu!`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.lu!). In-place version. Input matrix gets overwritten with L and U.
    - Or call LAPACK wrapper function [`LAPACK.getrf!`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.getrf!) directly.
    - Other LU-related LAPACK wrapper functions: [`getrs`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.getrs!), [`gesv`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.gesv!), [`gesvx`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.gesvx!), [`trtri`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.trtri!) (inverse of triangular matrix), [`trtrs`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.trtrs!).

In [46]:
# LAPACK : Linear Algebra Package
    # GETRF : partial pivoting 이용한 LU decomposition 함수
    # DGESV : solve() in R
# LinearAlgebra.lu! : LAPACK.getrf! 불러서 사용 (메모리 할당 X)###

In [ ]:
############################################################

In [92]:
A = [2.0 -4.0 2.0; 4.0 -9.0 7.0; 2.0 1.0 3.0]
A

3×3 Matrix{Float64}:
 2.0  -4.0  2.0
 4.0  -9.0  7.0
 2.0   1.0  3.0

In [93]:
P1 = [0 1 0; 1 0 0; 0 0 1] # p=2
P1*A

3×3 Matrix{Float64}:
 4.0  -9.0  7.0
 2.0  -4.0  2.0
 2.0   1.0  3.0

In [94]:
M1 = [1 0 0; -0.5 1 0; -0.5 0 1]
M1*(P*A)

3×3 Matrix{Float64}:
 4.0  -9.0   7.0
 0.0   0.5  -1.5
 0.0   5.5  -0.5

In [95]:
P2 = [1 0 0; 0 0 1; 0 1 0] # p=3
P2*( M1*(P*A) )

3×3 Matrix{Float64}:
 4.0  -9.0   7.0
 0.0   5.5  -0.5
 0.0   0.5  -1.5

In [97]:
M2 = [1 0 0; 0 1 0; 0 -1/11 1]
U = M2 * ( P2*( M1*(P*A) ) )

3×3 Matrix{Float64}:
 4.0  -9.0   7.0
 0.0   5.5  -0.5
 0.0   0.0  -1.45455

In [100]:
P = P2*P1

3×3 Matrix{Int64}:
 0  1  0
 0  0  1
 1  0  0

In [103]:
M = M2*M1

3×3 Matrix{Float64}:
  1.0        0.0        0.0
 -0.5        1.0        0.0
 -0.454545  -0.0909091  1.0

In [ ]:
############################################################

In [ ]:
A = [2.0 -4.0 2.0; 4.0 -9.0 7.0; 2.0 1.0 3.0]
A

In [58]:
using LinearAlgebra

# second argument indicates partial pivoting (default) or not
alu = lu(A)
typeof(alu)

LU{Float64, Matrix{Float64}}

In [59]:
fieldnames(typeof(alu))

(:factors, :ipiv, :info)

In [60]:
alu.L # unit lower triangle

3×3 Matrix{Float64}:
 1.0  0.0        0.0
 0.5  1.0        0.0
 0.5  0.0909091  1.0

In [64]:
alu.U

3×3 Matrix{Float64}:
 4.0  -9.0   7.0
 0.0   5.5  -0.5
 0.0   0.0  -1.45455

In [66]:
alu.p

3-element Vector{Int64}:
 2
 3
 1

1. k=1 column 에서 $a_{11}, a_{21}, a_{31}$ 중 가장 큰 4를 갖는 $2^{nd}$ row 를 $1^{st}$ row로 옮긴다.
2. $M_1$을 곱해 k=1 column에 대한 기본행연산을 해준 후, 
3. 연산이 끝난 행렬 기준, k=2 column 에서 $a_{22}, a_{32}$ 중 큰 5.5를 갖는 $3^{rd}$ row를 $2^{nd}$ row로 옮긴다.

$\vdots$

In [68]:
alu.P

3×3 Matrix{Float64}:
 0.0  1.0  0.0
 0.0  0.0  1.0
 1.0  0.0  0.0

In [23]:
alu.L * alu.U

3×3 Matrix{Float64}:
 4.0  -9.0  7.0
 2.0   1.0  3.0
 2.0  -4.0  2.0

In [24]:
A[alu.p, :] # alu.P * A

3×3 Matrix{Float64}:
 4.0  -9.0  7.0
 2.0   1.0  3.0
 2.0  -4.0  2.0

In [25]:
# this is doing two triangular solves, 2n^2 flops
alu \ b

3-element Vector{Float64}:
 2.000000000000001
 1.0
 2.9999999999999996

In [26]:
det(A) # this does LU!
# LU를 한 다음 det를 구하는 순서

-32.0

In [27]:
# alu 는 이미 LU가 돼있는 상태
det(alu) # this is cheap

-32.0

In [28]:
inv(A) # this does LU!
# LU를 한 다음 inv를 구하는 순서

3×3 Matrix{Float64}:
  1.0625  -0.4375  0.3125
 -0.0625  -0.0625  0.1875
 -0.6875   0.3125  0.0625

In [29]:
inv(alu) # this is cheap

3×3 Matrix{Float64}:
  1.0625  -0.4375  0.3125
 -0.0625  -0.0625  0.1875
 -0.6875   0.3125  0.0625

## Further reading

* Sections II.5.2 and II.5.3, [Computational Statistics](https://link.springer.com/book/10.1007%2F978-0-387-98144-4) by James Gentle (2010).

* Chapter 2, [Applied Numerical Linear Algebra](https://doi.org/10.1137/1.9781611971446) by James W. Demmel (1997).

* Chapter 3, [Matrix Computation](https://www.amazon.com/Computations-Hopkins-Studies-Mathematical-Sciences/dp/1421407949/ref=sr_1_1?keywords=matrix+computation+golub&qid=1567157884&s=gateway&sr=8-1) by Gene Golub and Charles Van Loan (2013).

<img src="https://images-na.ssl-images-amazon.com/images/I/41f5vxegABL._SY344_BO1,204,203,200_.jpg" width="250" align="center"/>

## Acknowledgment

Many parts of this lecture note is based on [Dr. Hua Zhou](http://hua-zhou.github.io)'s 2019 Spring Statistical Computing course notes available at <http://hua-zhou.github.io/teaching/biostatm280-2019spring/index.html>.